# Working with Streaming Data

"Streaming data" is data that is continuously generated, often by some external source like a remote website, a measuring device, or a simulator. This kind of data is common for financial time series, web server logs,  scientific applications, and many other situations. 

The HoloViews ``Stream`` system provides a way to push arbitrary content to a ``DynamicMap`` callback, constructing a plot that updates over time.  This system was already discussed in the user guide sections [Responding to Events](11-Responding_to_Events.ipynb) and [Custom Interactivity](12-Custom_Interactivity.ipynb), but those examples showed streaming changes to plot metadata like zoom ranges, not to the underlying data.  Here, we will show how the HoloViews ``Pipe`` and ``Buffer`` streams can be used to work with streaming *data sources* as well. Apart from streaming directly in HoloViews we will also explore working with streaming data coordinated by the separate [``streamz``](http://matthewrocklin.com/blog/work/2017/10/16/streaming-dataframes-1) library from Matt Rocklin, which can make working with complex streaming pipelines much simpler.

*NOTE: To follow along with how it works, this example should be run one cell at a time in a Jupyter notebook.  Viewing a static copy from a website or the result of "Run All" will not show the dynamic updates provided by streaming.*

In [ ]:
import time
import numpy as np
import pandas as pd
import holoviews as hv

from holoviews.streams import Pipe, Buffer

import streamz
import streamz.dataframe

hv.extension('bokeh')

## Pipe

A ``Pipe`` allows data to be pushed into a DynamicMap callback to change a visualization, just like the streams in the [Responding to Events](./11-Responding_to_Events.ipynb) user guide were used to push changes to metadata that controlled the visualization. A ``Pipe`` can be used to push data of any type and make it available to a ``DynamicMap`` callback. Since all ``Element`` types accept ``data`` of various forms we can use ``Pipe`` to push data to an ``Element`` through a DynamicMap.

We can take advantage of the fact that most Elements can be instantiated without providing any data, so we declare the the ``Pipe`` with an empty list, declare the ``DynamicMap``, providing the pipe as a stream, which will dynamically draw a ``VectorField`` :

In [ ]:
pipe = Pipe(data=[])
vector_dmap = hv.DynamicMap(hv.VectorField, streams=[pipe])
vector_dmap.redim.range(x=(-1, 1), y=(-1, 1))

<img class="gif" src="http://assets.holoviews.org/gifs/guides/user_guide/Streaming_Data/pipe_vectorfield.gif"></img>

Having set up this ``VectorField`` tied to a ``Pipe`` we can start pushing data to it varying the orientation of the VectorField:

In [ ]:
x,y  = np.mgrid[-10:11,-10:11] * 0.1
sine_rings  = np.sin(x**2+y**2)*np.pi+np.pi
exp_falloff = 1/np.exp((x**2+y**2)/8)

for i in np.linspace(0, 1, 25):
    time.sleep(0.1)
    pipe.send([x,y,sine_rings*i, exp_falloff])

Since ``Pipe`` is completely general the data can be any custom type it provides a completely general mechanism to stream structured or unstructured data. Due to this generality it cannot provide some of the more complex features provided by the ``Buffer`` stream.

## Buffer

While ``Pipe`` provides a general solution for piping arbitrary data to ``DynamicMap`` callback,  ``Buffer`` on the other hand provides a very powerful means of working with streaming tabular data, defined as pandas dataframes, arrays or dictionaries of columns (as well as ``StreamingDataFrame`` we will cover later). ``Buffer`` automatically accumulates the last ``N`` rows of the tabular data, where ``N`` is defined by the ``backlog``.

The ability to accumulate data allows performing operations on a recent history of data, while plotting backends (such as bokeh) can optimize plot updates by sending just the latest patch.

#### A simple example

To initialize a ``Buffer`` we have to provide an example dataset which defines the columns and dtypes of the data we will be streaming. Next we define the ``backlog`` to keep the last 100 rows of data. If the data is a DataFrame we can specify whether we will also want to use the ``DataFrame`` ``index``. In this case we will simply define that we want to plot a ``DataFrame`` of 'x' and 'y' positions and a 'count' as ``Points`` and ``Curve`` elements:

In [ ]:
example = pd.DataFrame({'x': [], 'y': [], 'count': []}, columns=['x', 'y', 'count'])
dfstream = Buffer(example, backlog=100, index=False)
curve_dmap = hv.DynamicMap(hv.Curve, streams=[dfstream])
point_dmap = hv.DynamicMap(hv.Points, streams=[dfstream])

After applying some styling we will display an ``Overlay`` of the dynamic ``Curve`` and ``Points``

In [ ]:
point_opts = {'plot': dict(color_index='count', xaxis=None, yaxis=None),
              'style': dict(line_color='black', size=5)}
curve_dmap.opts(style=dict(line_width=1, color='black')) * point_dmap.opts(**point_opts)

<img class="gif" src="http://assets.holoviews.org/gifs/guides/user_guide/Streaming_Data/brownian.gif"></img>

Now that we have set up the ``Buffer`` and defined a ``DynamicMap`` to plot the data we can start pushing data to it. We will define a simple function which simulates brownian motion by accumulating x, y positions. We can ``send`` data through the ``hv.streams.Buffer`` directly.

In [ ]:
def gen_brownian():
    x, y, count = 0, 0, 0
    while True:
        x += np.random.randn()
        y += np.random.randn()
        count += 1
        yield pd.DataFrame([(x, y, count)], columns=['x', 'y', 'count'])

brownian = gen_brownian()
for i in range(200):
    dfstream.send(next(brownian))

Finally we can clear the data on the stream and plot using the ``clear`` method:

In [ ]:
dfstream.clear()

## Streamz

Now that we have discovered what the ``Pipe`` and ``Buffer`` streams can do it's time to use them together with the ``streamz`` library. Streamz helps you build pipelines to manage continuous streams of data. It is simple to use in simple cases, but also supports complex pipelines that involve branching, joining, flow control, feedback and more. Here we will mostly focus on connecting streamz output to HoloViews Streams so for more detail consult the [streamz documentation](https://streamz.readthedocs.io/en/latest/).

#### Using streamz.Stream and Pipe

Let's start with a fairly simple example:

1. Declare a ``streamz.Stream`` and a ``Pipe`` object and connect them into a pipeline into which we can push data. 
2. Use a ``sliding_window`` of 10, which will first wait for 10 sets of stream updates to accumulate. At that point and for every subsequent update, it will apply ``pd.concat`` to combine the most recent 10 updates into a new dataframe.
3. Use the ``sink`` method on the ``streamz.Stream`` to ``send`` the resulting collection of 10 updates to ``Pipe``.
4. Declare a ``DynamicMap`` that takes the sliding window of concatenated DataFrames and displays it using a ``Scatter`` Element.
5. Color the ``Scatter`` points by their 'count' and set a range, then display:

In [ ]:
point_source = streamz.Stream()
pipe = Pipe(data=[])
point_source.sliding_window(20).map(pd.concat).sink(pipe.send)
scatter_dmap = hv.DynamicMap(hv.Scatter, streams=[pipe])

After set up our streaming pipeline we can again display it:

In [ ]:
opts = dict(color_index='count', bgcolor='black')
scatter_dmap.opts(plot=opts).redim.range(y=(-4, 4))

<img class="gif" src="http://assets.holoviews.org/gifs/guides/user_guide/Streaming_Data/streamz1.gif"></img>

There is now a pipeline, but initially this plot will be empty, because no data has been sent to it. To see the plot update, let's use ``stream.emit`` and send small chunks of random pandas DataFrames to our plot:

In [ ]:
for i in range(100):
    df = pd.DataFrame({'x': np.random.rand(100), 'y': np.random.randn(100), 'count': i},
                      columns=['x', 'y', 'count'])
    point_source.emit(df)

#### Using StreamingDataFrame and StreamingSeries

The streamz library provides StreamingDataFrame and StreamingSeries as a powerful way to easily work with streaming tabular data, this makes it perfectly suited to work with ``Buffer``. With the ``StreamingDataFrame`` we can easily stream data, apply computations such as cumulative and rolling statistics and then visualize the data with HoloViews.

The ``streamz.dataframe`` module provides a ``Random`` utility that generates a ``StreamingDataFrame`` emitting random data with a certain frequency at a specified interval. The ``example`` attribute lets us see the structure and dtypes of the data we can expect:

In [ ]:
simple_sdf = streamz.dataframe.Random(freq='10ms', interval='100ms')
print(simple_sdf.index)
simple_sdf.example.dtypes

Since the ``StreamingDataFrame`` provides a pandas-like API, we can specify operations on the data directly. In this example we subtract a fixed offset and then compute the cumulative sum, giving us a randomly drifting timeseries. We can then pass the x-values of this dataframe to the HoloViews ``Buffer`` and supply ``hv.Curve`` as the ``DynamicMap`` callback to stream the data into a HoloViews ``Curve``:

In [ ]:
%%opts Curve [width=500 show_grid=True]
sdf = (simple_sdf-0.5).cumsum()
hv.DynamicMap(hv.Curve, streams=[Buffer(sdf.x)])

<img class="gif" src="http://assets.holoviews.org/gifs/guides/user_guide/Streaming_Data/streamz3.gif"></img>

The ``Random`` StreamingDataFrame will asynchronously emit events until it is stopped, which we can do by calling the ``stop`` method.

In [ ]:
simple_sdf.stop()

#### Making use of the StreamingDataFrame API

So far we have only computed the cumulative sum, but the ``StreamingDataFrame`` actually has an extensive API that lets us run a broad range of streaming computations on our data. For example, let's apply a rolling mean to our x-values with a window of 500ms and overlay it on top of the 'raw' data:

In [ ]:
%%opts Curve [width=500 show_grid=True]
source_df = streamz.dataframe.Random(freq='5ms', interval='100ms')
sdf = (source_df-0.5).cumsum()
raw_dmap = hv.DynamicMap(hv.Curve, streams=[Buffer(sdf.x)])
smooth_dmap = hv.DynamicMap(hv.Curve, streams=[Buffer(sdf.x.rolling('500ms').mean())])

raw_dmap.relabel('raw') * smooth_dmap.relabel('smooth')

<img class="gif" src="http://assets.holoviews.org/gifs/guides/user_guide/Streaming_Data/streamz4.gif"></img>

In [ ]:
source_df.stop()

#### Controlling the backlog

By default the ``Buffer`` accumulates a ``backlog`` of 1000 samples. In many cases this is overkill, but we can specify a shorter (or longer) backlog value to control how much history we accumulate, often depending on the element type. Another helpful trick is to use the ``functools.partial`` utility to declare the Element Dimensions.

In [ ]:
from functools import partial

multi_source = streamz.dataframe.Random(freq='5ms', interval='100ms')
sdf = (multi_source-0.5).cumsum()
hv.DynamicMap(hv.Table, streams=[Buffer(sdf.x, backlog=10)]) +\
hv.DynamicMap(partial(hv.BoxWhisker, kdims=[], vdims=['x']), streams=[Buffer(sdf.x, backlog=100)])

<img class="gif" src="http://assets.holoviews.org/gifs/guides/user_guide/Streaming_Data/streamz5.gif"></img>

Here the given stream ``sdf`` is being consumed by a table showing a short backlog (where only the items visible in the table need to be kept), along with a plot computing averages and variances over a longer backlog (100 items).

#### Updating multiple cells

Since a ``StreamingDataFrame`` will emit data until it is stopped, we can subscribe multiple plots across different cells to the same stream.  Here, let's add a ``Scatter`` plot of the same data stream as in the preceding cell:

In [ ]:
hv.DynamicMap(hv.Scatter, streams=[Buffer(sdf.x)])

<img class="gif" src="http://assets.holoviews.org/gifs/guides/user_guide/Streaming_Data/streamz6.gif"></img>

Stopping the stream will now stop updates to all three of these DynamicMaps:

In [ ]:
multi_source.stop()

## Operations

As we discovered above, the ``Buffer`` lets us define a backlog window defining how many samples we want to accumulate. We can use this to our advantage and apply an operation over this backlog window. In this example we declare a ``Dataset`` and then apply the ``histogram`` operation to compute a ``Histogram`` over the specified ``backlog`` window:

In [ ]:
hist_source = streamz.dataframe.Random(freq='5ms', interval='100ms')
sdf = (hist_source-0.5).cumsum()
dmap = hv.DynamicMap(hv.Dataset, streams=[Buffer(sdf.x, backlog=500)])
hv.operation.histogram(dmap, dimension='x')

<img class="gif" src="http://assets.holoviews.org/gifs/guides/user_guide/Streaming_Data/streamz7.gif"></img>

In [ ]:
hist_source.stop()

#### Datashading

The same approach will also work for the datashader operation letting us datashade the entire ``backlog`` window even if we make it very large such as 1 million samples:

In [ ]:
%%opts RGB [width=600]
from holoviews.operation.datashader import datashade
from bokeh.palettes import Blues8
large_source = streamz.dataframe.Random(freq='100us', interval='200ms')
sdf = (large_source-0.5).cumsum()
dmap = hv.DynamicMap(hv.Curve, streams=[Buffer(sdf.x, backlog=1000000)])
datashade(dmap, streams=[hv.streams.PlotSize], normalization='linear', cmap=Blues8)

<img class="gif" src="http://assets.holoviews.org/gifs/guides/user_guide/Streaming_Data/streamz8.gif"></img>

In [ ]:
large_source.stop()

## Asynchronous updates

In most cases, instead of pushing Stream updates manually from the same Python process, you'll want the object to update asynchronously as new data arrives. Since both Jupyter and Bokeh server run on Tornado, we can use the tornado ``IOLoop`` in both cases to define a non-blocking co-routine that can push data to our stream whenever it is ready.

In [ ]:
%%opts Curve [width=600]
from tornado.ioloop import IOLoop
from tornado import gen

buffer = Buffer(np.zeros((0, 2)))

@gen.coroutine
def f():
    for x in range(500):
        gen.sleep(0.1)
        buffer.send(np.array([[x, np.random.rand()]]))
        
IOLoop.current().add_callback(f)
hv.DynamicMap(hv.Curve, streams=[buffer]).redim.range(y=(0, 1))

<img class="gif" src="http://assets.holoviews.org/gifs/guides/user_guide/Streaming_Data/streamz2.gif"></img>

Here the plot should update 100 times as before, but now via the Tornado IOLoop which will not block other interactions and work in the notebook.

## Real examples

Using the ``Pipe`` and ``Buffer`` streams we can create complex streaming plots very easily. In addition to the toy examples we presented in this guide it is worth looking at looking at some of the examples using real, live, streaming data.

* The [streaming_psutil](http://holoviews.org/gallery/apps/bokeh/stream_psutil.html) bokeh app is one such example which display CPU and memory information using the ``psutil`` library  (install with ``pip install psutil`` or ``conda install psutil``)

<img class="gif" src="http://assets.holoviews.org/gifs/guides/user_guide/Streaming_Data/streamz9.gif"></img>

As you can see, streaming data works like streams in HoloViews in general, flexibly handling changes over time under either explicit control or governed by some external data source.